# Lesson 3: Training Models in Low Carbon Regions

* Import libraries to train a model locally.

In [81]:
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

* Create a dataset.

This is an example from the [Advanced Learning Algorithms](https://www.coursera.org/learn/advanced-learning-algorithms) course.

In [82]:
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

* Create the model.

In [83]:
# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

In [84]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

* Train the model.

In [69]:
model.fit(
    X_train,y_train,
    epochs=200
)

Epoch 1/200
4/4 [==============================] - 0s 2ms/step - loss: 1.6501
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 1.4961
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3781
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2871
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2286
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1793
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1408
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1069
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0763
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0474
Epoch 11/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0222
Epoch 12/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9973
Epoch 13/200
4/4 [==============================] - 0s 1ms/st

### Running on Google Cloud

We will use Google Cloud's ML platform, Vertex AI. To run this code on Vertex AI, we first need to:
1. Import and initialize the Vertex AI Python SDK
2. Write the ML training code to a file
3. Configure and submit a training job that runs our training code.

* Step 1. Initialize Vertex AI.

In [85]:
from helper import authenticate
CREDENTIALS, PROJECT_ID = authenticate()

In [86]:
from google.cloud import aiplatform

In [87]:
aiplatform.init(project=PROJECT_ID,
                credentials=CREDENTIALS,
                )

* Step 2. Write the ML training code to a file.

In [88]:
%%writefile task.py

# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)

Overwriting task.py


In [77]:
ls

L3_Training_Models.ipynb*  __pycache__/  helper.py*  task.py*


In [89]:
cat task.py


# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)


* Define the Custom Trainin Job.

> Note: Check here to find more information about [Prebuilt containers for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)

> Note: Check here to find more information about [Carbon free energy for Google Cloud regions](https://cloud.google.com/sustainability/region-carbon)

In [90]:
# Choose a location (with carbon free energy available for this example)
REGION = 'us-central1'

* Create a store bucket.

In [91]:
import random
import string

In [92]:
def generate_uuid(length: int = 8) -> str:
    return "".join(
        random.choices(
            string.ascii_lowercase + string.digits, 
            k=length))

UUID = generate_uuid()

In [93]:
# The unique identifier
UUID

'bqpwcsf9'

In [94]:
from google.cloud import storage

In [95]:
storage_client = storage.Client(project = PROJECT_ID,
                                credentials = CREDENTIALS)

In [96]:
bucket_name = f'carbon-course-bucket-{UUID}'

In [97]:
bucket_name

'carbon-course-bucket-bqpwcsf9'

In [98]:
bucket = storage_client.bucket(bucket_name)

In [99]:
bucket.create(location=REGION)

In [100]:
# Create the CustomTrainingJob
job = aiplatform.CustomTrainingJob(
    display_name='dlai-course-example',
    script_path='task.py',
    container_uri='us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-14.py310:latest',
    staging_bucket=f'gs://{bucket_name}',
    location=REGION,
)

In [101]:
model = job.run()

Training script copied to:
gs://carbon-course-bucket-bqpwcsf9/aiplatform-2024-07-18-06-41-39.500-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://carbon-course-bucket-bqpwcsf9/aiplatform-custom-training-2024-07-18-06-41-39.500
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3727316689851777024?project=362161713137
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1619438550195896320?project=362161713137
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJ

In [107]:
# Delete the bucket to clean up resources
bucket.delete(force=True)